# Foundational LLM

## Using LCEL

> `BasePromptTemplate`, `BaseLanguageModel` and `BaseOutputParser` all implement the `Runnable` interface and are designed to be piped into one another, making LCEL composition very easy:

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

In [9]:
prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

In [10]:
runnable = prompt | ChatOpenAI() | StrOutputParser()

In [11]:
runnable.invoke({'product': 'colorful socks'})

'Colorful Threads'

In [16]:
for s in runnable.stream({'product': 'colorful socks'}):
    print(s, end='', flush=True)

SpectrumSock Co.

## Legacy LLMChain

An `LLMChain` consists of a `PromptTemplate` and a language model (either an LLM or chat model).<br>
It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.
> 输入键以及内存键 都可以用来格式化提示模板

In [17]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [18]:
prompt_template = "What is a good name for a company that makes {product}?"

### \_\_call\_\_ 返回字典

In [19]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

### run 返回字符串

In [22]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain.run("colorful socks")

'\n\nSocktastic!'

### predict 输入k=v,返回字符串

predict is similar to run method except that the input keys are specified as keyword arguments instead of a Python dict.

In [23]:
# Single input example
llm_chain.predict(product="colorful socks")

'\n\nSocktastic!'

In [24]:
# Multiple inputs example
template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0))

llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'

## Additional ways of running LLMChain

### apply: 多个输入,多个输入,均为字典
> **apply allows you run the chain against a list of inputs:**

In [20]:
input_list = [{"product": "socks"}, {"product": "computer"}, {"product": "shoes"}]
llm_chain.apply(input_list)

[{'text': '\n\nSocktastic!'},
 {'text': '\n\nTechCore Solutions.'},
 {'text': '\n\nFootwear Factory.'}]

### generate: 返回LLMResult

generate is similar to apply, except it return an `LLMResult` instead of string. LLMResult often contains useful generation such as `token usages` and `finish reason`.
> 什么是finish reason ? <br>
```python
> generation_info={'finish_reason': 'stop', 'logprobs': None}
```

In [21]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSocktastic!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nTechCore Solutions.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nFootwear Factory.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'prompt_tokens': 36, 'completion_tokens': 19, 'total_tokens': 55}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('216f98dc-bdf1-443a-9e8c-7543275b9f3f')), RunInfo(run_id=UUID('dae879cb-fbe6-4c21-8a6c-f9c7b7943e5a')), RunInfo(run_id=UUID('6f5e2d6b-cd61-4659-a4dc-51539f2d832d'))])

## Parsing the outputs

In [25]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [26]:
output_parser = CommaSeparatedListOutputParser()

In [27]:
template = """List all the colors in a rainbow"""

In [28]:
prompt = PromptTemplate(
    template=template, input_variables=[], output_parser=output_parser
)

In [29]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [30]:
llm_chain.predict()

'\n\nRed, orange, yellow, green, blue, indigo, violet'

In [31]:
llm_chain.predict_and_parse()

/opt/conda/envs/preventloss/lib/python3.9/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

## Initialize from string

> You can also construct an LLMChain from a string template directly.

In [32]:
template = """Tell me a {adjective} joke about {subject}."""

In [33]:
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [34]:
llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'